# MAPPO Integration

This runs the integrated MAPPO algorithm with the patrolling zoo.

In [7]:
%reload_ext autoreload
%autoreload 2
from onpolicy.scripts.train.train_patrolling import get_config, parse_args, main

import os
os.environ["WANDB__SERVICE_WAIT"] = "300"

In [8]:
parser = get_config()
all_args = parse_args([], parser)

all_args.experiment_name = "separatePolicies"
all_args.env_name = "Patrolling"
all_args.user_name = "anthony-goeckner"

all_args.num_agents = 2
all_args.agent_speed = 10.0
all_args.observe_method = "adjacency"
all_args.async_actions = False

all_args.graph_name = "4nodes"
all_args.graph_file = f"patrolling_zoo/env/{all_args.graph_name}.graph"
all_args.num_env_steps = 10e4 * 5 #total number of steps
all_args.episode_length = 200 #number of steps in a training episode
all_args.max_cycles = all_args.episode_length #number of steps in an environment episode

all_args.algorithm_name = "rmappo"
all_args.use_recurrent_policy = True
all_args.use_naive_recurrent_policy = False
all_args.use_centralized_V = True
all_args.use_gae = True
all_args.use_gae_amadm = False
all_args.share_policy = False

all_args.n_rollout_threads = 25
all_args.cuda = True
all_args.cuda_idx = 0

all_args.use_wandb = True

In [9]:
main([], parsed_args = all_args)

u are choosing to use rmappo, we set use_recurrent_policy to be True
choose to use gpu...



 Env Patrolling Algo rmappo Exp separatePolicies updates 0/100 episodes, total num timesteps 5000/500000.0, FPS 1383.

average episode rewards is 27044.219970703125

 Env Patrolling Algo rmappo Exp separatePolicies updates 1/100 episodes, total num timesteps 10000/500000.0, FPS 1299.

average episode rewards is 20814.120483398438

 Env Patrolling Algo rmappo Exp separatePolicies updates 2/100 episodes, total num timesteps 15000/500000.0, FPS 1357.

average episode rewards is 23844.105529785156

 Env Patrolling Algo rmappo Exp separatePolicies updates 3/100 episodes, total num timesteps 20000/500000.0, FPS 1398.

average episode rewards is 26580.52978515625

 Env Patrolling Algo rmappo Exp separatePolicies updates 4/100 episodes, total num timesteps 25000/500000.0, FPS 1422.

average episode rewards is 20637.124633789062

 Env Patrolling Algo rmappo Exp separatePolicies updates 5/100 episodes, total num timesteps 30000/500000.0, FPS 1438.

average episode rewards is 25154.879760742188


agent0/actor_grad_norm,▃█▅▃▃▄▄▃▆▃▅▄▁▂▄▅▄▆▂▄▄▄▄▅▃▄▃▂▂▃▂▂▃▂▃▃▂▂▁▁
agent0/critic_grad_norm,█▆▃▃▁▂▁▁▄▁▂▂▁▂▁▁▂▁▁▂▁▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
agent0/dist_entropy,███▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁▁
agent0/policy_loss,▄▂▅▄▃▃▃▄▂▅▁▄▇▆▂▃▃▄▅▁▃▄▃▄▆▅▆▇▇▆▇▅▄▆▅▆█▇▆█
agent0/ratio,▅▃▅▃▄▃▁▄▅▃▃▅▄▄▃▂█▁▅▅▅▅▃▃▅▄▂▃▄▃▄▃▃▅▆▄▃▄▅▄
agent0/value_loss,▅▂▃▇▁▅▃▂█▄▅█▃▇▄▂▁▃▃▁▃▆▇▇▇▆▆▇▅▄▅▅▅▅▄▅▄▂▄▄
agent1/actor_grad_norm,█▂▃▃▆▃▅▃▂▄▃▄▄▃▅▆▆▆▄▄▃▂▄▂▄▂▃▄▃▂▅▄▁▃▅▆▆▃▂▂
agent1/critic_grad_norm,█▇▂▂▂▃▃▃█▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▂▂▁▂▂▂▁▁▁▁▁▂▂▁▁
agent1/dist_entropy,██████▇▇▇▇▆▆▆▅▅▄▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂
agent1/policy_loss,▁▆▆▄▂▄▃▆▆▃▂▃▄▃▄▄▄▅▆▅▅▇▅▅▆▇▅▆▆▆▅▄█▅▅▅▅▆▆▇
agent1/ratio,█▄▃▄▆▇▆▆▁▅▇█▆▆▆▇▄▆▃▅▄▄▅▆▅▆▅▆▅▅▅▅▇▆▆▇▄▅▅▆
